# Creating the Channel Funding Transaction

In this section, we’ll create a lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use Test Framwork the created on `chapter 0 - lightning node keys derivation`.

In [5]:
%run "../chapter 0 - lightning node keys derivation/lightning node keys derivation.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f


In [6]:
from functions.test_framework.key import generate_bip340_key_pair
from functions.test_framework.address import program_to_witness
from functions.test_framework.script import tagged_hash
from functions.bip0327.reference import individual_pk, key_agg, get_xonly_pk, key_sort, apply_tweak, nonce_gen, nonce_agg, SessionContext, sign, partial_sig_verify, partial_sig_agg, schnorr_verify

### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [ ]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey.get_bytes())
print(f"Alice funding pubkey: {alice_funding_pubkey.get_bytes().hex()}")
print(f"Alice funding privkey: {alice_funding_privkey.get_bytes().hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Alice key pair for using to spend the htlc timeout transaction (chapter 5)
alice_sweeper_privkey, alice_sweeper_pubkey = generate_bip340_key_pair()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_sweeper_address = program_to_witness(0x01, alice_sweeper_pubkey.get_bytes())
print(f"Alice sweeper pubkey: {alice_sweeper_pubkey.get_bytes().hex()}")
print(f"Alice sweeper privkey: {alice_sweeper_privkey.get_bytes().hex()}")
print(f"Alice sweeper address: {alice_sweeper_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey.get_bytes())
print(f"Bob funding pubkey: {bob_funding_pubkey.get_bytes().hex()}")
print(f"Bob funding privkey: {bob_funding_privkey.get_bytes().hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
sweeper_initial_fund = 0.1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund, alice_sweeper_address: sweeper_initial_fund})
print(f"Transaction creating Alice UTXO: {txid_to_spend}")
rawtx_json = node.getrawtransaction(txid_to_spend, True)
print(json.dumps(rawtx_json, indent=2, default=str))

expected_spk = (b"\x51" + b"\x20" + alice_funding_pubkey.get_bytes()).hex()
# Get the index of the Alice output to spend
index_to_spend = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == expected_spk
)

sweeper_index = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == (b"\x51" + b"\x20" + alice_sweeper_pubkey.get_bytes()).hex()
)

print(f"Alice transaction to spend: {txid_to_spend}:{index_to_spend}")
print(f"Sweeper transaction to spend: {txid_to_spend}:{sweeper_index}")

result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: bd69395a813b1407fda9065b6c01bfea9f403465cd5a24858e2228c2f824a77d
Alice funding privkey: ad01b94e4b772ae2294f5aabd450bb051879773d7f45b337e91d2342a1d4591c
Alice funding address: bcrt1ph45njk5p8v2q0ldfqedkcqdla205qdr9e4dzfpvwyg5v97py5a7s6nrg09
Bob funding pubkey: fedf5840aca3fd81d8c48a5336b6cc01c80b5a4d47ddd90e6196f2a82f8f36de
Bob funding privkey: e256f386185e685ff637723c676f28a3e46a26fdfaa11eccdb46097377b7f4cd
Bob funding address: bcrt1plm04ss9v507crkxy3ffnddkvq8yqkkjdglwajrnpjme2stu0xm0q2ukumj
Transaction creating Alice UTXO: d1c5dc1affa2f9bdb105cafb315eee3706c1bbd3dc32b0b3d9f9a8fc6e9452fc
{
  "txid": "d1c5dc1affa2f9bdb105cafb315eee3706c1bbd3dc32b0b3d9f9a8fc6e9452fc",
  "hash": "5f121599d3b7ec9015f984b7b1e345b0cab4039d8c685cd4c14cf52fe13b636a",
  "version": 2,
  "size": 248,
  "vsize": 197,
  "weight": 788,
  "locktime": 102,
  "vin": [
    {
      "txid": "8c2aeb69390787e719c479bc5783e1997e62c364702ed61b3776a93b8562802e",
      "vout": 2,
      "scriptSig": {
    

## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                                                            +---------+
    |           |--(1)---------- open_channel (next local nonce) ----------->|         |
    |           |<-(2)---------- accept_channel (next local nonce) ----------|         |
    |           |                                                            |         |
    |   Alice   |--(3)------  funding_created (partial sig) (jit nonce)----->|   Bob   |
    |           |<-(4)------  funding_signed (partial sig) (jit nonce)-------|         |
    |           |                                                            |         |
    |           |--(5)---------- channel_ready (next local nonce) ---------->|         |
    |           |<-(6)---------- channel_ready (next local nonce) -----------|         |
    +-----------+                                                            +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [8]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey.get_bytes())
print(f"Alice Change pubkey: {alice_change_pubkey.get_bytes().hex()}")
print(f"Alice Change privkey: {alice_change_privkey.get_bytes().hex()}")
print(f"Alice Change address: {alice_change_address}")


Alice Change pubkey: 19fd48c1514bb3795415bfd3c09dac6c0e983fe2f8c84bb4286c21cb220b357b
Alice Change privkey: 3c1c16ba5ef377b04dafc28665b26c692b034b4cd9949545be53218e6cc350b9
Alice Change address: bcrt1pr8753s23fwehj4q4hlfup8dvds8fs0lzlryyhdpgdssukgstx4as8y80z7


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed. [BIP 341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#cite_ref-22-0) states: "If the spending conditions do not require a script path, the output key should commit to an unspendable script path instead of having no script path. This can be achieved by computing the output key point as Q = P + int(hashTapTweak(bytes(P)))G. And [BIP 86](https://github.com/bitcoin/bips/blob/master/bip-0086.mediawiki) define the following: 

```
internal_key:       lift_x(derived_key)
32_byte_output_key: internal_key + int(HashTapTweak(bytes(internal_key)))G
```

As our internal key is the agregated public key of Alice and Bob, we have:


    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [9]:
# Create a new Alice key pair for channel funding musig2 output
privkey_alice_musig2, pubkey_alice_musig2 = generate_bip340_key_pair()
print(f"Alice MuSig2 pubkey: {pubkey_alice_musig2.get_bytes().hex()}")
print(f"Alice MuSig2 privkey: {privkey_alice_musig2.get_bytes().hex()}")

# Create a new Bob key pair for channel funding musig2 output
privkey_bob_musig2, pubkey_bob_musig2 = generate_bip340_key_pair()
print(f"Bob MuSig2 pubkey: {pubkey_bob_musig2.get_bytes().hex()}")
print(f"Bob MuSig2 privkey: {privkey_bob_musig2.get_bytes().hex()}")

# Generate a 2-of-2 tweaked aggregate MuSig2 key using the pubkeys form Alice and Bob
sorted_pubkeys = key_sort([pubkey_alice_musig2.get_bytes(bip340=False), pubkey_bob_musig2.get_bytes(bip340=False)])
key_agg_ctx = key_agg(sorted_pubkeys)
agg_pubkey = get_xonly_pk(key_agg_ctx)

# Compute the BIP86 tweak
bip86_tweak = tagged_hash("TapTweak", agg_pubkey)
tweaked_ctx = apply_tweak(key_agg_ctx, bip86_tweak, is_xonly=True)

# Final Taproot output key (x-only) to put in the P2TR scriptPubKey
agg_pubkey_tweaked = get_xonly_pk(tweaked_ctx)

print(f"Aggregate musig2 pubkey: {agg_pubkey_tweaked.hex()}")

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked)
print(f"Channel funding musig2 output address: {channel_funding_musig_address}")

Alice MuSig2 pubkey: f4f25af95cd78ae38467b043d30c4811cc4d5ea0c47e243c34682b3365d28147
Alice MuSig2 privkey: eec50454805fd075477c449e5dda5f68c29f87455009466f89b5443ab0f1d786
Bob MuSig2 pubkey: f5099ed1404802de2be4b52f9f34e5792964d46ca90d0eacf6bdd764220d110d
Bob MuSig2 privkey: 08687858e89ef57c289ff0aeeb417f6eece312456d0711380f3873f65f42940e
Aggregate musig2 pubkey: dcdcf5b0fb760d506daaa28c897dbd4f9c4d2216f165eb1bea57873fb824b6c7
Channel funding musig2 output address: bcrt1pmnw0tv8mwcx4qmd252xgjldaf7wy6gsk79j7kxl227rnlwpykmrs3hvass


#### The channel funding transaction

##### The unsigned transaction

In [10]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
channel_value_sat = int(float("0.01") * 100000000)
channel_value = channel_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
channel_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked) + agg_pubkey_tweaked

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
alice_change_value_sat = int(float(alice_initial_fund) * 100000000) - channel_value_sat - tx_fee_sat
alice_change_value = alice_change_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_change_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey.get_bytes()) + alice_change_pubkey.get_bytes()

outputs = (
    channel_value
    + varint_len(channel_spk)
    + channel_spk
    + alice_change_value
    + varint_len(alice_change_spk)
    + alice_change_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001fc52946efca8f9d9b3b032dcd3bbc10637ee5e31fbca05b1bdf9a2ff1adcc5d10000000000ffffffff0240420f0000000000225120dcdcf5b0fb760d506daaa28c897dbd4f9c4d2216f165eb1bea57873fb824b6c7949de6050000000022512019fd48c1514bb3795415bfd3c09dac6c0e983fe2f8c84bb4286c21cb220b357b00000000
{
  "txid": "61dd28757a0c95ff7b9ca7bb86b322927a1db27111e9aa9f00ba42679f5a8d92",
  "hash": "61dd28757a0c95ff7b9ca7bb86b322927a1db27111e9aa9f00ba42679f5a8d92",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "d1c5dc1affa2f9bdb105cafb315eee3706c1bbd3dc32b0b3d9f9a8fc6e9452fc",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 dcdcf5b0fb760d506daaa28c897dbd4f9c4d2216f165eb1bea57873fb824b6c7",
        "desc": "rawtr(dcdcf5b0fb760d506daaa28c897dbd4f9c4d2216f165eb1bea57873f

##### The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [11]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey.get_bytes()) + alice_funding_pubkey.get_bytes()

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
msg = tag_hash + tag_hash + sig_msg
sighash = sha256(msg).digest()


##### Signing the sighash

In [12]:
signature = alice_funding_privkey.sign_schnorr(sighash) 

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [13]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

funding_channel_txid = node.sendrawtransaction(signed_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("funding_channel_txid: ", funding_channel_txid)

signed tx:  02000000000101fc52946efca8f9d9b3b032dcd3bbc10637ee5e31fbca05b1bdf9a2ff1adcc5d10000000000ffffffff0240420f0000000000225120dcdcf5b0fb760d506daaa28c897dbd4f9c4d2216f165eb1bea57873fb824b6c7949de6050000000022512019fd48c1514bb3795415bfd3c09dac6c0e983fe2f8c84bb4286c21cb220b357b01403d190cf5d9c5e1721c2b1a0b21c2ce3758a066f1115d03de217fcbfe9c9b94f03490c966a76431123701848490d7775f09b457a63e4791dbb2f5b5afd2b1664a00000000
{
  "txid": "61dd28757a0c95ff7b9ca7bb86b322927a1db27111e9aa9f00ba42679f5a8d92",
  "hash": "ce6f3940df52ea7f78f2c085ee62f7437730175867016496360b15a86f02b5c5",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "d1c5dc1affa2f9bdb105cafb315eee3706c1bbd3dc32b0b3d9f9a8fc6e9452fc",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3d190cf5d9c5e1721c2b1a0b21c2ce3758a066f1115d03de217fcbfe9c9b94f03490c966a76431123701848490d7775f09b457a63e4791dbb2f5b5afd2b

## Summary

This chapter demonstrated how to create a Lightning channel funding transaction from scratch:

### Funding Transaction Structure
**Input**: Single UTXO from Alice (1 BTC)

**Outputs**:
1. **Channel Funding Output (0.01 BTC)**: 2-of-2 MuSig2 multisig
   - Aggregate pubkey: `Q = P_agg + t*G` where `t = TaggedHash("TapTweak", P_agg)`
   - Uses BIP86 tweak (no script path, unspendable script commitment)
   - Requires both parties' signatures to spend
2. **Change Output (0.9899 BTC)**: Returns remaining funds to Alice

### MuSig2 Signing Process
- Pubkeys sorted using KeySort (BIP327)
- Aggregate key computed with KeyAgg
- Both parties generate nonces and exchange pubnonces
- Each creates partial signature using sign algorithm
- Partial signatures verified with partial_sig_verify
- Final signature aggregated with partial_sig_agg
- Result is a single 64-byte Schnorr signature

### Key Takeaway
The funding transaction creates a secure 2-of-2 multisig output on-chain that both parties must cooperate to spend. This output serves as the anchor for all future commitment transactions, ensuring neither party can unilaterally spend the channel funds.